# MNIST Fashion Dataset
### Predicting the type of clothing using various TensorFlow models


In [ ]:
# Set up notebook headers
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from metrics import *
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.callbacks import History
from tensorflow.keras.datasets import fashion_mnist

%matplotlib inline
%load_ext autoreload
%autoreload 2

#### Data Collection
Fashion MNIST is a dataset within TensorFlow that contains 70,000 grayscale images of clothing items categorized into 10 classes, such as T-shirts, dresses, and shoes. Each image is 28x28 pixels, making it a popular benchmark for training and testing machine learning models for image classification tasks.

#### Loading Raw Data
Read in Fashion MNIST dataset directly via TensorFlow.

In [ ]:
# Load the Fashion MNIST dataset
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

# Display the shape of the datasets
print("Fashion MNIST Dataset Dimensions")
print("Shape of training images:", train_images.shape)
print("Shape of training labels:", train_labels.shape)
print("Shape of test images:", test_images.shape)
print("Shape of test labels:", test_labels.shape)

#### Visualise Raw Data
Have a peek at some images to see how they are presented.

In [ ]:
# Visualise the first 10 images in the training set
plt.figure(figsize = (10, 6))
for i in range(10):
    plt.subplot(2, 5, i + 1)  
    plt.imshow(train_images[i])
    plt.title(f"Label: {train_labels[i]}")
    plt.axis('off')  
plt.tight_layout()
plt.suptitle("Visualising Fashion MNIST Dataset (10 Examples)", fontsize = 16)
plt.show()

The labels are provided as integers, so we will need to map them to the corresponding clothing categories for the final predictions.

In [ ]:
classes = ["T-shirt/top", "Trouser", "Pullover", "Dress", "Coat", "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"]

#### Preprocess Data
Let's reshape the data and ensure pixel values in the images are normalised so the models can interpret them correctly. Whilst the labels don't necessarily need to be one-hot encoded, let's do so anyway for completeness. Finally, we'll need to split the entire dataset of 70,000 images into training, validation, and test sets using a 75/15/10 split, respectively.

In [ ]:
# Combine the current train and test sets to create one dataset
images = np.concatenate([train_images, test_images], axis = 0)
labels = np.concatenate([train_labels, test_labels], axis = 0)

# Preprocess the data
num_classes = np.max(labels) + 1
images_mod = images.reshape((-1, images.shape[1], images.shape[1], 1)).astype('float32') / 255.0

In [ ]:
# Split into training, validation, and test sets
x_train, x_temp, y_train, y_temp = train_test_split(images_mod, labels, test_size = 0.25)
x_val, x_test, y_val, y_test = train_test_split(x_temp, y_temp, test_size = 0.4)

In [ ]:
# Print shape of datasets
print("Fashion MNIST Modified Dataset Dimensions")
print("Shape of training images:", x_train.shape)
print("Shape of training labels:", y_train.shape)
print("Shape of validation images:", x_val.shape)
print("Shape of validation labels:", y_val.shape)
print("Shape of test images:", x_test.shape)
print("Shape of test labels:", y_test.shape)

#### Model Creation
We will be building various TensorFlow models and testing their performance on the test set to decide what the optimal configuration should be.

##### Model 1: Baseline

Build a simple neural network with 2 hidden layers and the Adam optimiser.

In [ ]:
# Set up model by adding layers sequentially
base = models.Sequential()
base.add(layers.Flatten(input_shape = (x_train.shape[1], x_train.shape[1], 1)))
base.add(layers.Dense(units = 256, activation = 'relu'))
base.add(layers.Dense(units = 128, activation = 'relu'))

# Add output layer
base.add(layers.Dense(units = num_classes, activation = 'softmax'))

# Compile model
base.compile(optimizer = 'adam',
             loss = 'sparse_categorical_crossentropy',
             metrics = ['accuracy'])

# Define a History callback to record training metrics
history = History()

# Fit model on training data
base.fit(x_train, y_train, batch_size = 32, epochs = 10, callbacks = [history],  validation_data = (x_val, y_val))

# Print model summary
base.summary()

Let's look at the training progress over all epochs.

In [ ]:
# Plot training progress for training and validation data
epochs = np.arange(1, 11)
training_progress(epochs, history.history['accuracy'], history.history['loss'], history.history['val_accuracy'], history.history['val_loss'])

View performance metrics after predicting on the test set.

In [ ]:
# Predict on test set
base_preds = base.predict(x_test)

# Extract confidence scores for predicted labels
confidence_scores = np.max(base_preds, axis = 1)

# Extract predicted labels
y_pred = np.argmax(base_preds, axis = 1)

In [ ]:
# Plot confusion matrix
confusion_matrix_ad(y_test, y_pred, num_classes, classes)

In [ ]:
# Plot distribution of confidence scores
confidence_distribution(confidence_scores)

In [ ]:
# Display classification report
classification_report_ad(y_test, y_pred)

**Comments**: 

##### Model 2: Deep Neural Network (DNN) - No Regularisation

Build a deep neural network with 3 hidden layers and the RMSProp optimiser.

In [ ]:
# Set up model by adding layers sequentially
dnn = models.Sequential()
dnn.add(layers.Flatten(input_shape = (x_train.shape[1], x_train.shape[1], 1)))

# Hidden layers with L2 regularization
dnn.add(layers.Dense(256, activation = 'relu'))
dnn.add(layers.Dense(128, activation = 'relu'))
dnn.add(layers.Dense(64, activation = 'relu'))

# Output layer
dnn.add(layers.Dense(num_classes, activation = 'softmax'))

# Compile the model
dnn.compile(optimizer = 'rmsprop',
            loss = 'sparse_categorical_crossentropy',
            metrics = ['accuracy'])

# Define a History callback to record training metrics
history = History()

# Fit model on training data
dnn.fit(x_train, y_train, batch_size = 32, epochs = 10, callbacks = [history],  validation_data = (x_val, y_val))

# Print model summary
dnn.summary()

Let's look at the training progress over all epochs.

In [ ]:
# Plot training progress for training and validation data
training_progress(epochs, history.history['accuracy'], history.history['loss'], history.history['val_accuracy'], history.history['val_loss'])

View performance metrics after predicting on the test set.

In [ ]:
# Predict on test set
dnn_preds = dnn.predict(x_test)

# Extract confidence scores for predicted labels
confidence_scores = np.max(dnn_preds, axis = 1)

# Extract predicted labels
y_pred = np.argmax(dnn_preds, axis = 1)

In [ ]:
# Plot confusion matrix
confusion_matrix_ad(y_test, y_pred, num_classes, classes)

In [ ]:
# Plot distribution of confidence scores
confidence_distribution(confidence_scores)

In [ ]:
# Display classification report
classification_report_ad(y_test, y_pred)

**Comments**: 

##### Model 3: Deep Neural Network (DNN) - With Regularisation

Build a deep neural network with 3 hidden layers, the RMSProp optimiser, and L2-Regularisation.

In [ ]:
# Set up model by adding layers sequentially
dnn_reg = models.Sequential()
dnn_reg.add(layers.Flatten(input_shape = (x_train.shape[1], x_train.shape[1], 1)))

# Hidden layers with L2 regularization
dnn_reg.add(layers.Dense(256, activation = 'relu', kernel_regularizer = regularizers.l2(0.01)))
dnn_reg.add(layers.Dense(128, activation = 'relu', kernel_regularizer = regularizers.l2(0.01)))
dnn_reg.add(layers.Dense(64, activation = 'relu', kernel_regularizer = regularizers.l2(0.01)))

# Output layer
dnn_reg.add(layers.Dense(num_classes, activation = 'softmax'))

# Compile the model
dnn_reg.compile(optimizer = 'rmsprop',
            loss = 'sparse_categorical_crossentropy',
            metrics = ['accuracy'])

# Define a History callback to record training metrics
history = History()

# Fit model on training data
dnn_reg.fit(x_train, y_train, batch_size = 32, epochs = 10, callbacks = [history],  validation_data = (x_val, y_val))

# Print model summary
dnn_reg.summary()

Let's look at the training progress over all epochs.

In [ ]:
# Plot training progress for training and validation data
training_progress(epochs, history.history['accuracy'], history.history['loss'], history.history['val_accuracy'], history.history['val_loss'])

View performance metrics after predicting on the test set.

In [ ]:
# Predict on test set
dnn_reg_preds = dnn_reg.predict(x_test)

# Extract confidence scores for predicted labels
confidence_scores = np.max(dnn_reg_preds, axis = 1)

# Extract predicted labels
y_pred = np.argmax(dnn_reg_preds, axis = 1)

In [ ]:
# Plot confusion matrix
confusion_matrix_ad(y_test, y_pred, num_classes, classes)

In [ ]:
# Plot distribution of confidence scores
confidence_distribution(confidence_scores)

In [ ]:
# Display classification report
classification_report_ad(y_test, y_pred)

**Comments**: 

##### Model 4: Convolutional Neural Network (CNN)

Build a convolutional neural network with 3 convolutional layers, the SGD optimiser, and a dropout of 0.1 between the last 2 convolutional layers.

In [ ]:
# Set up model by adding layers sequentially
cnn = models.Sequential()
cnn.add(layers.Conv2D(32, (3, 3), activation = 'relu', input_shape = (x_train.shape[1], x_train.shape[1], 1)))
cnn.add(layers.MaxPooling2D((2, 2)))

# Add convolutional layers for localised feature extraction
cnn.add(layers.Conv2D(64, (3, 3), activation = 'relu'))
cnn.add(layers.MaxPooling2D((2, 2)))
cnn.add(layers.Dropout(0.1))

cnn.add(layers.Conv2D(64, (3, 3), activation = 'relu'))
cnn.add(layers.Flatten())
cnn.add(layers.Dropout(0.1))
cnn.add(layers.Dense(64, activation = 'relu'))
cnn.add(layers.Dense(num_classes, activation = 'softmax'))

# Compile model
cnn.compile(optimizer = 'sgd',
            loss = 'sparse_categorical_crossentropy',
            metrics = ['accuracy'])

# Define a History callback to record training metrics
history = History()

# Fit model on training data
cnn.fit(x_train, y_train, batch_size = 32, epochs = 10, callbacks = [history],  validation_data = (x_val, y_val))

# Print model summary
cnn.summary()

Let's look at the training progress over all epochs.

In [ ]:
# Plot training progress for training and validation data
training_progress(epochs, history.history['accuracy'], history.history['loss'], history.history['val_accuracy'], history.history['val_loss'])

View performance metrics after predicting on test set.

In [ ]:
# Predict on test set
cnn_preds = cnn.predict(x_test)

# Extract confidence scores for predicted labels
confidence_scores = np.max(cnn_preds, axis = 1)

# Extract predicted labels
y_pred = np.argmax(cnn_preds, axis = 1)

In [ ]:
# Plot confusion matrix
confusion_matrix_ad(y_test, y_pred, num_classes, classes)

In [ ]:
# Plot distribution of confidence scores
confidence_distribution(confidence_scores)

In [ ]:
# Display classification report
classification_report_ad(y_test, y_pred)

**Comments**: 

##### Model 5: Recurrent Neural Network (RNN)

Build a recurrent neural network with 2 LSTM layers, the Adam optimiser, and L2-Regularisation.

In [ ]:
# Set up model by adding layers sequentially
rnn = models.Sequential()
rnn.add(layers.Reshape((x_train.shape[1], x_train.shape[1]), input_shape = (x_train.shape[1], x_train.shape[1], 1)))

# LSTM layers with L2 regularization
rnn.add(layers.LSTM(256, return_sequences = True, kernel_regularizer = regularizers.l2(0.01)))
rnn.add(layers.LSTM(128, kernel_regularizer = regularizers.l2(0.01)))

# Output layer
rnn.add(layers.Dense(num_classes, activation = 'softmax'))

# Compile the model
rnn.compile(optimizer = 'adam',
            loss = 'sparse_categorical_crossentropy',
            metrics = ['accuracy'])

# Define a History callback to record training metrics
history = History()

# Fit model on training data
rnn.fit(x_train, y_train, batch_size = 32, epochs = 10, callbacks = [history],  validation_data = (x_val, y_val))

# Print model summary
rnn.summary()

Let's look at the training progress over all epochs.    

In [ ]:
# Plot training progress for training and validation data
training_progress(epochs, history.history['accuracy'], history.history['loss'], history.history['val_accuracy'], history.history['val_loss'])

View performance metrics after predicting on the test set.

In [ ]:
# Predict on test set
rnn_preds = rnn.predict(x_test)

# Extract confidence scores for predicted labels
confidence_scores = np.max(rnn_preds, axis = 1)

# Extract predicted labels
y_pred = np.argmax(rnn_preds, axis = 1)

In [ ]:
# Plot confusion matrix
confusion_matrix_ad(y_test, y_pred, num_classes, classes)

In [ ]:
# Plot distribution of confidence scores
confidence_distribution(confidence_scores)

In [ ]:
# Display classification report
classification_report_ad(y_test, y_pred)

**Comments**: 

##### Model 6: Residual Network (ResNet)

Build a residual neural network with 1 convolutional layer, 2 residual blocks, and the Adam optimiser.

In [ ]:
# Define Residual Block
def residual_block(x, filters, kernel_size):
    y = layers.Conv2D(filters, kernel_size, padding = 'same')(x)
    y = layers.BatchNormalization()(y)
    y = layers.Activation('relu')(y)
    
    y = layers.Conv2D(filters, kernel_size, padding = 'same')(y)
    y = layers.BatchNormalization()(y)
    
    # Skip connection
    if x.shape[-1] != filters:
        x = layers.Conv2D(filters, kernel_size = (1, 1), padding = 'same')(x)
    y = layers.add([x, y])
    y = layers.Activation('relu')(y)
    return y

# Define ResNet model
def ResNet(input_shape, num_classes):
    inputs = layers.Input(shape = input_shape)
    
    # Initial convolutional layer
    x = layers.Conv2D(64, (7, 7), strides = (2, 2), padding = 'same')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.MaxPooling2D((3, 3), strides = (2, 2), padding = 'same')(x)
    
    # Residual blocks
    x = residual_block(x, filters = 64, kernel_size = (3, 3))
    x = residual_block(x, filters = 64, kernel_size = (3, 3))
    
    x = layers.GlobalAveragePooling2D()(x)
    outputs = layers.Dense(num_classes, activation = 'softmax')(x)
    
    model = models.Model(inputs, outputs)
    return model

# Create ResNet model
input_shape = x_train.shape[1:]
resnet = ResNet(input_shape, num_classes)

# Compile the model
resnet.compile(optimizer = 'adam',
                     loss = 'sparse_categorical_crossentropy',
                     metrics = ['accuracy'])

# Fit model on training data
resnet.fit(x_train, y_train, batch_size = 32, epochs = 10, callbacks = [history],  validation_data = (x_val, y_val))

# Print model summary
resnet.summary()

Let's look at the training progress over all epochs.    

In [ ]:
# Plot training progress for training and validation data
training_progress(epochs, history.history['accuracy'], history.history['loss'], history.history['val_accuracy'], history.history['val_loss'])

View performance metrics after predicting on the test set.

In [ ]:
# Predict on test set
resnet_preds = resnet.predict(x_test)

# Extract confidence scores for predicted labels
confidence_scores = np.max(resnet_preds, axis = 1)

# Extract predicted labels
y_pred = np.argmax(resnet_preds, axis = 1)

In [ ]:
# Plot confusion matrix
confusion_matrix_ad(y_test, y_pred, num_classes, classes)

In [ ]:
# Plot distribution of confidence scores
confidence_distribution(confidence_scores)

In [ ]:
# Display classification report
classification_report_ad(y_test, y_pred)

**Comments**: 

#### Results

#### Ambiguous Images
Using the best model, find the 10 most ambiguous images in the test set based on predicted confidence scores.